In [1]:
import json

with open("../results/gpt-4-vision-preview_input_2_shot_0_20231221_101231/add_no_image/detail.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)
    
with open("../data/problem_v1.2.0_20231217.json","r",encoding='utf-8') as f:
    raw_raw_data = json.load(f)

In [2]:
#define the customed result files and parameters here...

details_filelist = r"""   
gpt-4-vision-preview_input_2_shot_0_20231221_101231/add_no_image  
qwen-vl_input_2_shot_0_it_20231225_083611   
gemini-pro-vision_input_2_shot_0_20231224_004723/add_no_image   
moss_input_0_shot_0_20231226_070654  
visualglm_input_2_shot_0_it_20231226_070315
gemini-pro_input_0_shot_0_20231223_141324
gpt-3.5-turbo-0613_input_0_shot_0_20231220_143731                
viscpm_input_2_shot_0_it_bi_20231228_194002
gpt-4-1106-preview_input_0_shot_0_20231220_214000
""".split()
# details_filelist

details_weights = [63.7,38.9,53.7,32.6,31.1,52.2,40.1,33.4,50.2]
assert len(details_filelist) == len(details_weights)

In [3]:
import os

detailed_score_ensemble_list = []
for filedir in details_filelist:
    file = os.path.join("..", "results", filedir, "detail.json")
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)
        detailed_score_ensemble_list.append(data)


def judge_score_for_a_problem(problem_key):
    sumed = 0
    for filedir, det, weight in zip(
        details_filelist, detailed_score_ensemble_list, details_weights
    ):
        question_images = len(det.get("image_list", []))
        prob_det = det[problem_key]
        if prob_det["total_score"] == 0:
            continue
        if question_images and "input_0" in filedir:
            continue
        sumed += (1 - prob_det["score"] / prob_det["total_score"]) * weight
    return sumed

In [4]:
select_type_categories = "yw sx yy zz ls dl wl hx sw dx jk xc".split()

classified_list = [
    [[] for _ in range(len(select_type_categories))] for _ in range(3)
]  # (0,1,m)

multot = 0
# First Iteration: Generate metadata of all the problems
# metadata of each question: (transferred_id, type, model_score_criteria, comp result of diff & quality)
for key, problem in raw_data.items():
    source_key = key.rsplit('_',1)[0]
    problem_o = raw_raw_data[source_key]
    difficulty, quality = problem_o["difficulty"], problem_o["quality"]
    
    meta_triplet = (
        key,
        problem["type"],
        judge_score_for_a_problem(key),
        difficulty * 1.5 + quality,
    )
    question_images = len(problem.get("image_list", []))
    
    if question_images >= 2:
        question_images = 3
        multot += 1

    sub = key.split("_")[0]
    subs = "dx" if sub[:2] == "dx" else sub[-2:]

    classified_list[question_images ^ 1][select_type_categories.index(subs)].append(
        meta_triplet
    )  # [SinglePics, NonPics, MultiPics]

print(len(raw_data))
multot

18330


214

In [34]:
from typing import List, Tuple, Union
import random

type_list = ["单选", "多选", "填空", "解答", "其他"]
from functools import cmp_to_key


def absolute_cmp(x, y) -> int:
    return (x > y) - (x < y)


def sorting_criteria(
    s0: Tuple[Union[str, int, float]], s1: Tuple[Union[str, int, float]]
):
    if s0[1] != s1[1]:
        s0_index, s1_index = type_list.index(s0[1]), type_list.index(s1[1])
        return absolute_cmp(-s0_index, -s1_index)
    elif s0[2] != s1[2]:
        return absolute_cmp(s0[2], s1[2])
    else:
        return absolute_cmp(s0[-1], s1[-1])


import math

selection_matrix = [ [50,15,50], [30,10,5], [10,5,10]]
# selection_matrix = [
#     [20, 22, 24],
#     [20, 20, 24],
#     [20, 20, 24],
# ]  # number matrix. A precise selection will make the selection more accurate.
# (SI, NI, MI) x (middle_school, univ, society)


result_list = []

for row_index, (selection_row, metadatas_list) in enumerate(
    zip(selection_matrix, classified_list)
):
    for i, metadata_list in enumerate(metadatas_list):
        random.shuffle(metadata_list)
        metadata_list.sort(key=cmp_to_key(sorting_criteria), reverse=True)
        sc_total, mc_total, bf_total = [
            len(list(filter(lambda x: x[-3] == type_list[k], metadata_list)))
            for k in range(3)
        ]

        # [:single_total]
        neg_index = i - len(select_type_categories)
        if neg_index < -3:  # middleschool
            j = 0
        elif neg_index == -3:  # university
            j = 1
        else:
            j = 2
        tot_needed_in_this_section = selection_matrix[row_index][j]*2
        mc, bf = (
            math.ceil(tot_needed_in_this_section / 4),
            math.floor(tot_needed_in_this_section / 4),
        )

        # A better solution is splitting the list into 3 and perform sampling respectively.

        result_list.extend(
            metadata_list[sc_total : sc_total + min(mc, mc_total)]
        )  # MuC
        result_list.extend(
            metadata_list[sc_total + mc_total : sc_total + mc_total + min(bf, bf_total)]
        )  # BF
        result_list.extend(
            metadata_list[
                : min(
                    sc_total,
                    tot_needed_in_this_section - min(mc, mc_total) - min(bf, bf_total),
                )
            ]
        )  # SiC
        #TODO: sample might be done here?
        
len(result_list)

1629

In [35]:
random.shuffle(result_list)
result_list=result_list[:500]

with open("selected_hard_20231229.json","w",encoding='utf-8') as fw:
    json.dump(result_list,fw,ensure_ascii=False,indent=4)

In [36]:
import json

with open("../data/selected_hard_20231229.json","r",encoding='utf-8') as f:
    result_list = json.load(f)

In [37]:
with open("../data/selected_hard_list_20231229.json", "w", encoding="utf-8") as fw:
    json.dump(list(map(lambda x: x[0], result_list)), fw)

In [38]:
# Analysis on the correctness
import os

keys = map(lambda x: x[0], result_list)
# keys = result_list

input_file = (
    "../results/gpt-4-vision-preview_input_2_shot_0_20231221_101231/add_no_image/prediction.json"
)
with open(input_file, "r", encoding="utf-8") as f:
    preds = json.load(f)

hard_performance = {}
cnt = 0
for key in keys:
    if key in preds:
        hard_performance.update({key: preds[key]})
    else:
        # print(f"{key} missig in the preds")
        cnt += 1
        
print(cnt)

output_dir = f"../results/selected_analysis/{'/'.join(input_file.split('/')[2:-1])}"
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, input_file.split("/")[-1])
with open(output_file, "w", encoding="utf-8") as fw:
    json.dump(hard_performance, fw, ensure_ascii=False, indent=4)

0
